<a href="https://colab.research.google.com/github/SofiMich/DL/blob/main/T1_Michaelian_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#3. Regresión lineal con PyTorch

Para este ejercicio me voy a guiar en el notebook de APIs de Pytorch que vimos en clase con los ayudantes.

## 1 Preparación

### 1.1 Bibliotecas

In [ ]:
import math
import os
import random

import matplotlib.pyplot as plt
import matplotlib.axes as plt3d #para graficar en 3D

import numpy as np

import pandas as pd

# redes neuronales
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.utils.data import DataLoader, TensorDataset

### 1.2 Auxiliares

In [ ]:
# datos
URL = 'https://raw.githubusercontent.com/gibranfp/CursoAprendizajeProfundo/2023-1/data/califs/califs.csv'
data_dir = '../data'
filepath = os.path.join(data_dir, 'califs.csv')

def set_seed(seed=0):
    """Initializes pseudo-random number generators."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

## 2 Datos

Descargamos los datos.

In [ ]:
! mkdir {data_dir}
! wget -nc {URL} -O {filepath}

Cargamos los datos.

In [ ]:
df = pd.read_csv(filepath)

Graficamos los datos en 3D:

In [ ]:
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")

ax.scatter3D(df['prev'],df['horas'],df['calif'])
ax.set_xlabel('Calificación previa', fontweight ='bold')
ax.set_ylabel('Horas de estudio', fontweight ='bold')
ax.set_zlabel('Calificación', fontweight ='bold')

plt.show()

Creamos nuestros arreglos con los datos:

In [ ]:
x_trn = np.array(df.iloc[:,:2], dtype="float32")
y_trn = np.array(df.iloc[:,-1], dtype="float32")[..., np.newaxis]

x_trn = torch.tensor(x_trn)
y_trn = torch.tensor(y_trn)

print(x_trn.shape)
print(y_trn.shape)

Y los metemos a un tensor:

In [ ]:
ds = TensorDataset(x_trn, y_trn)
ds[0]

### 2.2 Cargador de datos


Para ver el funcionamiento de la tubería de datos imprimimos la forma de cada lote y su primer elemento.

In [ ]:
def build_dl(batch_size=16, shuffle=True):
    return DataLoader(ds, batch_size=batch_size, shuffle=True)

# creamos un DataLoader
dl = build_dl()

x, y = next(iter(dl))
print(f'x shape={x.shape} dtype={x.dtype}')
print(f'y shape={y.shape} dtype={y.dtype}')

In [ ]:
len(ds)

## 3 Entrenamiento
Para el entrenamiento vamos a usar el descenso por gradiente estocástico como optimizador:

 y el error cuadrático medio como función de pérdida:

In [ ]:
def train(model, dl, epochs=5, lr=1e-3):

    opt = optim.SGD(model.parameters(), lr)

    loss_hist = []

    for epoch in range(epochs):

        # historial
        loss = []
        
        # entrenamiento de una época
        for x, y_true in dl:
            # inferencia
            y_lgts = model(x)
            # calculamos de pérdida y exactitud
            p = F.mse_loss(y_lgts, y_true) 
            
            # vaciamos los gradientes
            opt.zero_grad()
            # retropropagamos
            p.backward()
            # actulizamos parámetros
            opt.step()

            # guardamos historial de pérdida
            loss.append(p.item() * 100)
            
        # imprimimos la pérdida de la época
        loss_hist.append(np.mean(loss))

        print(f'E{epoch:02} pérdida: [{loss_hist[-1]:6.2f}] ')
  
    return  loss_hist
        
def train_model(build_model, epochs=5):
    set_seed()
    dl = build_dl()
    model = build_model()
    perdida = train(model, dl, epochs)
    return perdida

## 4 Definición de la arquitectura

Para implementar arquitecturas, PyTorch define dos clases fundamentales.

* `nn.Module` define una red neuronal que internamente puede tener otras redes neuronales anidadas (o capas). Tres metodos importantes son:
  * `__init__(self, args)` es el inicilizador que define al objeto,
  * `fordward(x)` realizar predicción (hacia adelante),
  * `parameters(x)` regresa una lista de los parámetros (`nn.Parameter`) de la red y redes anidadas.


* `nn.Parameter` envuelve un tensor solo para marcarlo como parámetro y que sea regresado por `nn.Module.parameters(x)`.

### 4.1 Alto nivel 
Como es ua regresión lineal, sólo necesitamos una capa lineal sin fución de activación:

In [ ]:
def build_high():
    model = nn.Sequential(
        nn.Linear(2, 1)
    )
    return model

build_high()

### 4.2 Nivel medio
Lo mismo, solo usamos una capa lineal:

In [ ]:

class LinRegMed(nn.Module):

    # Inicializador:
    def __init__(self):
        # se llama al inicializador de la clase padre
        super().__init__()
        
        self.fc2 = nn.Linear(2, 1)
        

    # método para inferencia
    def forward(self, x):
        x = self.fc2(x)
        return x

def build_med():
    return LinRegMed()

build_med()

## 5. Entrenando modelos
Para entrenar los modelos usaré 100 épocas con una taza de aprendizaje de $1\times10^{-3}$

In [ ]:
perdidaHi = train_model(build_high,100)

In [ ]:
perdidaMed = train_model(build_med, 100)

Al comparar los dos modelos, vemos que dan resultados muy similares par la pérdida:

In [ ]:
plt.plot(range(100),perdidaHi, label='Alto ivel')
plt.plot(range(100),perdidaMed, label = 'Nivel medio')

plt.xlabel('época')
plt.ylabel('pérdida')
plt.show()

## 6. Obteniendo  parámetros

In [ ]:
high = build_high()
med = build_med()

In [ ]:
list(high.parameters())

In [ ]:
list(med.parameters())

Los parámetros estimados por las redes son similares, pero no son iguales

##7. Inferencia

Para hacer la inferencia sobre un alumno que estudió durante 12 horas y obtuvo 3 de caificación en un examen previo hacemos:

In [ ]:
med(torch.tensor([4.0,12.0]))

In [ ]:
high(torch.tensor([4.0,12.0]))

Dado que los parámetros que cada red aprendió son diferentes, sus predicciones son diferentes.

In [ ]:
y_predichaFinalMed = med(x_trn).detach().numpy().reshape(50,)

In [ ]:
fig = plt.figure(figsize = (10, 7))
ax = plt.axes(projection ="3d")

np.arange(5, 10, 0.1)
x_line = np.arange(5, 10, 5.0/10)
y_line = np.arange(8, 14, 6.0/10)
z_line = 0.6499*x_line + 0.2892*y_line

#ax.plot3D(x_line,y_line,z_line, color = 'red')
ax.plot_trisurf(df['prev'],df['horas'],y_predichaFinalMed)
ax.scatter3D(df['prev'],df['horas'],df['calif'])
ax.set_xlabel('Calificación previa', fontweight ='bold')
ax.set_ylabel('Horas de estudio', fontweight ='bold')
ax.set_zlabel('Calificación', fontweight ='bold')

plt.show()

In [ ]:
perdidaHi = train_model(build_high,100)